https://proceedings.mlr.press/v70/wang17h.html

https://random-walks.org/content/misc/gumbel/gumbel.html

In [1]:
import sys
import time, cProfile

In [2]:
import numpy as np
from numpy.random import rand
from numpy import sum, log, sqrt, exp # should I use functions from numpy or math? ans- al though math is 4 times faster, 
                                      # you'll have to use numpy because ur dealing with numpy arrays
from numpy import tile, unique, ones, zeros, diag
from numpy.linalg import cholesky, solve

In [3]:
import scipy as sp
from scipy.optimize import minimize

## Code

In [4]:
def sample_categorical(p, dx):
    # This function samples from a categorical distribution 
    
    '''
        How efficent is this sampling?
    '''

    rd = rand(dx)
    cur_p = 0
    z = np.zeros(dx)

    for i in range(len(p)):                                  
        cur_p = cur_p + p[i]                               
        z = z + np.logical_and(z==0, rd <= cur_p)*(i+1)
    
    return z

In [5]:
class hyp:
    def __init__(self, l , sigma0, sigma):
        hyp.l = l
        hyp.sigma0 = sigma0
        hyp.sigma = sigma

In [6]:
def computeKmm(xbar, l, sigma, sigma0):

    m = xbar.shape[0]
    xbar = xbar * tile(sqrt(l), (m, 1))
    #print('xbar times lbar:', xbar1)
    qbar = tile(sum(xbar**2, 1).reshape(m, 1), (1, m))
    #print('qbar: ', qbar1)
    distance = (qbar + qbar.T) - 2*(xbar @ xbar.T)
    #print('distance: ', distance)
    ret = sigma * exp(-0.5*distance) + np.identity(m) * (sigma0 + sigma * 1e-10) # or np.eye()
    return ret

In [7]:
def compute_gram(xx, hyp, z):   
    all_cat = unique(z).astype(int)
    K = 0
    for i in range(len(all_cat)):

        xbar = xx[:, z==all_cat[i]]
        lbar = hyp.l[z==all_cat[i]]
        sigbar = hyp.sigma[all_cat[i]-1]
        sigbar0 = hyp.sigma0[all_cat[i]-1]
        K = K + computeKmm(xbar, lbar, sigbar, sigbar0)
    
    return K

In [8]:
def solve_chol(L, B):
    # add mat-mul incompatibility warning
    alpha = solve(L.T, solve(L, B))
    return alpha

In [32]:
def compute_nlz(xx, yy, hyp, z):

    nlz = 0
    # for idx = 1:size(hyp.l,1)
    K = compute_gram(xx, hyp, z)
    L = cholesky(K)
    alpha = solve_chol(L, yy.reshape(20, 10))         # d shape
    nlz = nlz + (yy.T @ alpha)/2 + sum(log(diag(L)))

    return nlz

In [10]:
def compute_nlz_wrap(xx, yy, param, n_partition, z):

    l = exp(np.array([param[i.astype(int)-1] for i in z]))
    sigma = exp(param[n_partition:n_partition*2])
    sigma0 = exp(param[n_partition*2:n_partition*3])

    hyp_w = hyp(l, sigma, sigma0)

    nlz = compute_nlz(xx, yy, hyp_w, z)

    return nlz

In [11]:
def globalMinimization(target, xmin, xmax, guesses):

    d = xmin.shape[0]   
    gridSize = 1000
    print('d: ',d)
    print('gridSize: ',gridSize)

    Xgrid = tile(xmin, (gridSize, 1)) + tile((xmax - xmin), (gridSize, 1))* rand(gridSize, d)
    print('Xgrid shape: ', Xgrid.shape)
    Xgrid = Xgrid.T
    print('Xgrid shape: ', Xgrid.shape)

    y = ones(Xgrid.shape[1])
    for i in range(Xgrid.shape[1]):
        y[i] = target(Xgrid[:, i])

    minIndex = np.argmin(y)
    minVal = np.min(y)
    print(minVal)

    start = Xgrid[:, minIndex]
    bounds = [(min, max) for min,max in zip(xmin,xmax)]
    x = minimize(fun=nll ,x0=start, method='trust-constr', bounds=bounds)
    print(x.x)
    print(x.fun)
    return x.x, x.fun

In [18]:
l = exp(np.array([b_p[i.astype(int)-1] for i in z]))
sigma = exp(b_p[n_partition:n_partition*2])
sigma0 = exp(b_p[n_partition*2:n_partition*3])
hyp_l = hyp(l, sigma, sigma0)

In [ ]:
log_prob

array([-4.11043933, -4.11043933, -3.12952795, -3.67317758, -3.66260053])

In [13]:
# sample_addGP
dx = 5
Xmin = zeros(dx) # Xmin is a column vector indicating the lower bound of the search space
Xmax = ones(dx) # Xmax is a column vector indicating the upper bound of the search space


# f = sample_addGp(dx, dx, Xmin, Xmax)
# options.nK = 5

options = hyp(ones(dx)*50, ones(dx)*5, ones(dx)*1e-4)

# add_gpopt(f, Xmin, Xmax, 200, [], [], options)

In [14]:
def stybtang(x):
    y = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in x[i]:
            y[i] += j**4 - 16*(j**2) + 5*j
        y[i] = y[i]*(1/2)
    return y

In [15]:
xx = np.random.uniform(-4, 4, (20, 5))
yy = stybtang(xx)


In [17]:
# sampleStructPriors function
n_partition=5
hyp_dirichlet = np.ones(n_partition) # tunable
prob_partition = hyp_dirichlet / sum(hyp_dirichlet)
z = sample_categorical(prob_partition, n_partition)
nll = lambda param: compute_nlz_wrap(xx, yy, param, n_partition, z)
guesses = []
bounds = np.array([tile([0, 10], (n_partition, 1)), tile([-5, 2], (n_partition, 1)), 
                   tile([-10, 1], (n_partition, 1))]).reshape(3*n_partition, 2) # why are we using these ranges? # [0, 10], [-5, 2], [-10, 1]
xmin = bounds[:, 0]                                                                                      
xmax = bounds[:, 1]
b_p, f_val = globalMinimization(nll, xmin, xmax, [])
print(z)

d:  15
gridSize:  1000
Xgrid shape:  (1000, 15)
Xgrid shape:  (15, 1000)
3385.0439192962344
[ 6.23196464e+00  6.50655783e-04  8.40397521e+00  9.13574904e+00
  4.95508502e-04  1.99977792e+00  1.99980154e+00 -3.41858936e-01
 -2.79210277e-01  1.99980206e+00 -4.77368035e+00  9.99675222e-01
 -7.19603146e-02 -3.99292388e+00  9.99746321e-01]
[2131.99101921]
[1. 2. 5. 2. 5.]


In [19]:
# def learn_partition(xx, yy, hyp, n_partition):

hyp_dirichlet = np.ones(n_partition) # tunable
prob_partition = hyp_dirichlet / sum(hyp_dirichlet)

z = sample_categorical(prob_partition, n_partition)
z_best = 0

N_gibbs = 10
gibbs_iter = np.floor(N_gibbs/2)

dx = xx.shape[1]

minnlz = sys.float_info.max
minnlz_nocon = sys.float_info.max

for i in range(N_gibbs):
    print('{} gibbs iteration'.format(i))

    for d in range(dx):
        print('     {} dimension'.format(d))
        log_prob = zeros(n_partition)
        nlz = zeros(n_partition)
        all_cat = unique(z.astype(int))
        other_cat = np.arange(1, n_partition+1)
        print('     other_cat',other_cat)

        for a in all_cat:
            other_cat = other_cat[other_cat != a]
            z[d] = a
            nlz[a-1] = compute_nlz(xx, yy, hyp, z)
            log_prob[a-1] = log(sum(z==a) + hyp_dirichlet[a-1]) - nlz[a-1] 

        if other_cat.size != 0:
            z[d] = other_cat[0]
            nlz[other_cat-1] = compute_nlz(xx, yy, hyp_l, z)
            log_prob[other_cat-1] = log(1 + hyp_dirichlet[other_cat-1]) - nlz[other_cat-1]

        print('     other_cat: ', other_cat)
        print('     nlz: ', nlz)
        print('     log_prob: ', log_prob)
        print('     z before: ', z)
        z[d] = np.argmax(log_prob)+1
        print('     z after: ', z)
        

        if minnlz_nocon > nlz[z[d].astype(int)-1]:
            z_best = z
            minnlz_nocon = nlz[z[d].astype(int)-1]
        
        if i == gibbs_iter-1:
            z = z_best

        print('     z_best', z_best)
        print('     minnlz_nocon', minnlz_nocon)
        print('\n')


0 gibbs iteration
     0 dimension
     other_cat [1 2 3 4 5]
     other_cat:  [2 4]
     nlz:  [2742.39891197 1956.50362114 3147.66472213 1956.50362114 3116.10314345]
     log_prob:  [-2741.30029968 -1955.81047396 -3146.56610984 -1955.81047396
 -3114.71684909]
     z before:  [2. 3. 5. 1. 5.]
     z after:  [2. 3. 5. 1. 5.]
     z_best [2. 3. 5. 1. 5.]
     minnlz_nocon 1956.5036211392332


     1 dimension
     other_cat [1 2 3 4 5]
     other_cat:  [4]
     nlz:  [2274.53184432 2293.38232035 1956.50362114 2213.87963085 2460.49723538]
     log_prob:  [-2273.43323203 -2292.28370806 -1955.81047396 -2213.18648367
 -2459.11094101]
     z before:  [2. 4. 5. 1. 5.]
     z after:  [2. 3. 5. 1. 5.]
     z_best [2. 3. 5. 1. 5.]
     minnlz_nocon 1956.5036211392332


     2 dimension
     other_cat [1 2 3 4 5]
     other_cat:  [4]
     nlz:  [1608.19404936 1611.04810661 1684.06064314 1576.25436347 1956.50362114]
     log_prob:  [-1607.09543708 -1609.94949432 -1682.96203085 -1575.56121629
 -195

In [20]:
def camel(x):
    y = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        y[i] = (4 - 2.1*x[i,0]**2 + (x[i,0]**4)/3 )*x[i,0]**2 + x[i,0]*x[i,1] + (-4 + 4*x[i,1]**2)*x[i,1]**2
    return y

In [21]:
xx = np.random.uniform(-2, 2, (20, 2))
yy = camel(xx)

In [22]:
n_partition=2
hyp_dirichlet = np.ones(n_partition) # tunable
prob_partition = hyp_dirichlet / sum(hyp_dirichlet)
z = sample_categorical(prob_partition, n_partition)
nll = lambda param: compute_nlz_wrap(xx, yy, param, n_partition, z)
guesses = []
bounds = np.array([tile([0, 10], (n_partition, 1)), tile([-5, 2], (n_partition, 1)), 
                   tile([-10, 1], (n_partition, 1))]).reshape(3*n_partition, 2) # why are we using these ranges? # [0, 10], [-5, 2], [-10, 1]
xmin = bounds[:, 0]                                                                                      
xmax = bounds[:, 1]
b_p, f_val = globalMinimization(nll, xmin, xmax, [])
print(z)

d:  6
gridSize:  1000
Xgrid shape:  (1000, 6)
Xgrid shape:  (6, 1000)
96.3265191154631


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_hessian_update_strategy.py:186: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


[1.09141289 9.08344421 2.         2.         1.         1.        ]
[74.6515096]
[2. 1.]


In [23]:
# def learn_partition(xx, yy, hyp, n_partition):

hyp_dirichlet = np.ones(n_partition) # tunable
prob_partition = hyp_dirichlet / sum(hyp_dirichlet)

z = sample_categorical(prob_partition, n_partition)
z_best = 0

N_gibbs = 10
gibbs_iter = np.floor(N_gibbs/2)

dx = xx.shape[1]

minnlz = sys.float_info.max
minnlz_nocon = sys.float_info.max

for i in range(N_gibbs):
    print('{} gibbs iteration'.format(i))

    for d in range(dx):
        print('     {} dimension'.format(d))
        log_prob = zeros(n_partition)
        nlz = zeros(n_partition)
        all_cat = unique(z.astype(int))
        other_cat = np.arange(1, n_partition+1)
        print('     other_cat',other_cat)

        for a in all_cat:
            other_cat = other_cat[other_cat != a]
            z[d] = a
            nlz[a-1] = compute_nlz(xx, yy, hyp, z)
            log_prob[a-1] = log(sum(z==a) + hyp_dirichlet[a-1]) - nlz[a-1] 

        if other_cat.size != 0:
            z[d] = other_cat[0]
            nlz[other_cat-1] = compute_nlz(xx, yy, hyp_l, z)
            log_prob[other_cat-1] = log(1 + hyp_dirichlet[other_cat-1]) - nlz[other_cat-1]

        print('     other_cat: ', other_cat)
        print('     nlz: ', nlz)
        print('     log_prob: ', log_prob)
        print('     z before: ', z)
        z[d] = np.argmax(log_prob)+1
        print('     z after: ', z)
        

        if minnlz_nocon > nlz[z[d].astype(int)-1]:
            z_best = z
            minnlz_nocon = nlz[z[d].astype(int)-1]
        
        if i == gibbs_iter-1:
            z = z_best

        print('     z_best', z_best)
        print('     minnlz_nocon', minnlz_nocon)
        print('\n')

0 gibbs iteration
     0 dimension
     other_cat [1 2]
     other_cat:  [1]
     nlz:  [ 74.65150947 131.57484853]
     log_prob:  [ -73.95836229 -130.47623624]
     z before:  [1. 2.]
     z after:  [1. 2.]
     z_best [1. 2.]
     minnlz_nocon 74.65150946625573


     1 dimension
     other_cat [1 2]
     other_cat:  []
     nlz:  [131.57484886  74.65150947]
     log_prob:  [-130.47623657  -73.95836229]
     z before:  [1. 2.]
     z after:  [1. 2.]
     z_best [1. 2.]
     minnlz_nocon 74.65150946625573


1 gibbs iteration
     0 dimension
     other_cat [1 2]
     other_cat:  []
     nlz:  [ 74.65150947 131.57484853]
     log_prob:  [ -73.95836229 -130.47623624]
     z before:  [2. 2.]
     z after:  [1. 2.]
     z_best [1. 2.]
     minnlz_nocon 74.65150946625573


     1 dimension
     other_cat [1 2]
     other_cat:  []
     nlz:  [131.57484886  74.65150947]
     log_prob:  [-130.47623657  -73.95836229]
     z before:  [1. 2.]
     z after:  [1. 2.]
     z_best [1. 2.]
     minn

In [27]:
def michNf(x):
    #Michalewicz function 
    s = 0
    m = 10
    n = len(x)
    for i in range(1, n+1):
        #s = s + sin(x[i-1])*(sin((i*(pow(x[i-1], 2)))/pi))^(2*m)
        s = s + np.sin(x[i-1])*np.sin((1 * x[i-1]**2) / np.pi)**(2*m)
    return -s

In [30]:
xx = np.random.uniform(-2, 2, (20, 10))
yy = michNf(xx)